# archive.org scraper

Import dependencies:

In [38]:
from selenium import webdriver
import time
import os

Get a list of the tars that I have already downloaded:

In [87]:
downloaded_tars = os.listdir('../data/2020_03_03_original_arxiv_tars')
downloaded_tars = [x.split('.')[0] for x in downloaded_tars]

Define scraping functions:

In [42]:
def initialize():
    # Construct browser and profile (to prevent download dialog)
    # https://stackoverflow.com/questions/25251583/downloading-file-to-specified-location-with-selenium-and-python
    profile = webdriver.FirefoxProfile()
    profile.set_preference('browser.download.folderList', 2)
    profile.set_preference('browser.download.manager.showWhenStarting', False)
    profile.set_preference('browser.download.dir', '/Volumes/BRIENNAKH/Thesis/data')
    profile.set_preference('browser.helperApps.neverAsk.saveToDisk', 'application/x-tar')
    return webdriver.Firefox(profile, executable_path='/usr/local/bin/geckodriver') # brew install chromedriver then see in terminal where it was installed to and paste this

def scroll(browser):
    # https://stackoverflow.com/questions/20986631/how-can-i-scroll-a-web-page-using-selenium-webdriver-in-python
    
    SCROLL_PAUSE_TIME = 0.5
    
    # Get scroll height
    last_height = browser.execute_script('return document.body.scrollHeight')
        
    while True:
        # Scroll down to bottom
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        
        # Calculate new scroll height and compare with last scroll height
        new_height = browser.execute_script('return document.body.scrollHeight')
        if new_height == last_height:
            break
        last_height = new_height

In [44]:
# Initialize browser
browser = initialize()

# Navigate to page with search results
browser.get('https://archive.org/search.php?query=arXiv_src&sort=-publicdate')

# Scroll until the results end
# scroll(browser)

In [74]:
# Collect all attributes
names = []
results = browser.find_elements_by_xpath('//div[@data-id]')
for result in results:
    name = result.get_attribute('data-id')
    if 'arXiv' in name:
        names.append(name)

In [86]:
print('Files in my downloaded tars that aren\'t in the archive: ' 
      + str(len(list(set(downloaded_tars) - set(names)))))

print('Files in the archive that aren\'t in my downloaded tars: ' 
     + str(len(list(set(names) - set(downloaded_tars)))))

Files in my downloaded tars that aren't in the archive: 2158
Files in the archive that aren't in my downloaded tars: 50


Go to the download page for each file:

In [76]:
for name in names:
    browser.get('https://archive.org/download/' + name)
    browser.find_element_by_partial_link_text('.tar').click()
    
    # need a way to wait until the file appears in download directory
    # but I am going to skip this since I would prefer to download from the S3 bucket,
    # it seems like archive.org may be missing quite a few files and it would not
    # be worth my time to try to figure out exactly which ones, and sync them up
    # when I can just get them straight from the source

NoSuchWindowException: Message: Browsing context has been discarded
